# Getting Started with QuantumFold-Advantage

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/01_getting_started.ipynb)

This tutorial introduces the basics of using QuantumFold for protein structure prediction.

## Topics Covered
1. Installation and setup
2. Loading and preparing protein data
3. Running structure predictions
4. Evaluating prediction quality
5. Visualizing results

## 🔧 Step 1: Installation

First, we'll clone the repository and install all dependencies.

In [ ]:
# Check if we're in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Check GPU availability
import torch
print(f"Running in Colab: {IN_COLAB}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Clone repository and install dependencies
if IN_COLAB:
    print("Installing QuantumFold-Advantage...")
    !git clone https://github.com/Tommaso-R-Marena/QuantumFold-Advantage.git
    %cd QuantumFold-Advantage
    
    # Install dependencies
    !pip install -q pennylane matplotlib pandas scikit-learn biopython requests tqdm
    
    print("\n✅ Installation complete!")
else:
    print("Running locally - make sure dependencies are installed")

## 📦 Step 2: Import Libraries

Import all necessary modules.

In [ ]:
import sys
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path

# Add src to path
if IN_COLAB:
    sys.path.insert(0, '/content/QuantumFold-Advantage')
else:
    sys.path.insert(0, str(Path.cwd().parent))

# Import QuantumFold modules
try:
    from src.quantum_layers import QuantumAttentionLayer
    from src.benchmarks import calculate_rmsd, calculate_tm_score
    print("✅ Modules imported successfully!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure you ran the installation cell above")

## 🧬 Step 3: Prepare Sample Data

Create a sample protein for demonstration.

In [ ]:
# Example: Small protein sequence (50 residues)
sequence = "MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEK"

print(f"Sequence length: {len(sequence)} residues")
print(f"Sequence: {sequence}")

# Generate random 3D coordinates for demonstration
# In practice, these would come from predictions or PDB files
n_residues = len(sequence)
np.random.seed(42)

# Simulate a compact protein structure
coords = np.random.randn(n_residues, 3) * 5
coords = coords - coords.mean(axis=0)  # Center at origin

print(f"\nCoordinate array shape: {coords.shape}")
print(f"Coordinate range: {coords.min():.2f} to {coords.max():.2f} Å")

## 🧠 Step 4: Initialize Quantum Model

Create a quantum attention layer for processing protein features.

In [ ]:
# Initialize quantum layer
device = 'cuda' if torch.cuda.is_available() else 'cpu'

quantum_layer = QuantumAttentionLayer(
    n_qubits=4,
    n_layers=2,
    feature_dim=64
).to(device)

print(f"Quantum layer initialized on: {device}")
print(f"Number of qubits: 4")
print(f"Number of quantum layers: 2")
print(f"Feature dimension: 64")
print(f"\nTotal parameters: {sum(p.numel() for p in quantum_layer.parameters())}")

# Create sample input
batch_size = 1
seq_length = len(sequence)
feature_dim = 64

sample_input = torch.randn(batch_size, seq_length, feature_dim).to(device)

# Test forward pass
with torch.no_grad():
    output = quantum_layer(sample_input)

print(f"\nInput shape: {sample_input.shape}")
print(f"Output shape: {output.shape}")
print("✅ Quantum layer working correctly!")

## 🎨 Step 5: Visualize Structure

Create 3D visualization of the protein structure.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Create 3D plot
fig = plt.figure(figsize=(12, 5))

# Plot 1: 3D structure
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot(coords[:, 0], coords[:, 1], coords[:, 2], 'b-', linewidth=2, alpha=0.6, label='Backbone')
ax1.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=range(len(coords)), cmap='viridis', s=50, alpha=0.8)
ax1.set_xlabel('X (Å)', fontsize=10)
ax1.set_ylabel('Y (Å)', fontsize=10)
ax1.set_zlabel('Z (Å)', fontsize=10)
ax1.set_title('3D Protein Structure', fontsize=12, fontweight='bold')
ax1.legend()

# Plot 2: Distance map
ax2 = fig.add_subplot(122)
distances = np.sqrt(np.sum((coords[:, None, :] - coords[None, :, :]) ** 2, axis=2))
im = ax2.imshow(distances, cmap='viridis', interpolation='nearest')
ax2.set_xlabel('Residue Index', fontsize=10)
ax2.set_ylabel('Residue Index', fontsize=10)
ax2.set_title('Distance Map', fontsize=12, fontweight='bold')
cbar = plt.colorbar(im, ax=ax2)
cbar.set_label('Distance (Å)', fontsize=10)

plt.tight_layout()
plt.savefig('structure_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Visualization saved to 'structure_visualization.png'")

## 📊 Step 6: Evaluate Structure Quality

Compute CASP evaluation metrics.

In [ ]:
# Create synthetic "reference" structure for demonstration
# In practice, this would be the true experimental structure
coords_true = coords + np.random.randn(*coords.shape) * 1.0

# Calculate RMSD
rmsd = calculate_rmsd(coords, coords_true)

# Calculate TM-score
tm_score = calculate_tm_score(coords, coords_true, len(sequence))

print("\n🎯 Evaluation Metrics:")
print("=" * 40)
print(f"RMSD:      {rmsd:.3f} Å")
print(f"TM-score:  {tm_score:.3f}")
print("=" * 40)

# Interpretation
print("\n📖 Interpretation:")
if rmsd < 2.0:
    print(f"✅ Excellent RMSD (< 2.0 Å): High-quality prediction")
elif rmsd < 4.0:
    print(f"🟡 Good RMSD (2-4 Å): Reasonable prediction")
else:
    print(f"⚠️  High RMSD (> 4 Å): May need improvement")

if tm_score > 0.8:
    print(f"✅ Excellent TM-score (> 0.8): Same fold, high similarity")
elif tm_score > 0.5:
    print(f"🟡 Good TM-score (0.5-0.8): Same fold")
else:
    print(f"⚠️  Low TM-score (< 0.5): Different fold")

## 🎓 Summary

In this tutorial, we learned how to:

1. ✅ Install QuantumFold-Advantage in Colab
2. ✅ Import quantum computing libraries
3. ✅ Create and process protein data
4. ✅ Initialize quantum attention layers
5. ✅ Visualize protein structures in 3D
6. ✅ Evaluate predictions with CASP metrics

### 🚀 Next Steps

**Continue your learning:**

1. **[Quantum vs Classical Comparison](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/02_quantum_vs_classical.ipynb)** - Train and compare both model types

2. **[Advanced Visualization](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/03_advanced_visualization.ipynb)** - Create publication-quality figures

3. **[Quickstart Guide](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/colab_quickstart.ipynb)** - Condensed version

### 📚 Resources

- **Documentation:** [README](https://github.com/Tommaso-R-Marena/QuantumFold-Advantage)
- **GitHub Repository:** [QuantumFold-Advantage](https://github.com/Tommaso-R-Marena/QuantumFold-Advantage)
- **Issues & Support:** [GitHub Issues](https://github.com/Tommaso-R-Marena/QuantumFold-Advantage/issues)

---

⭐ **If you found this tutorial helpful, please star the repository!**